These are the beginnings of snowflake, a browser based way to access linux containers. I want minimal dependencies, for maximal easy maintenence. 

In [ ]:
# First, I need a way to store used ip addresses, to avoid conflicts. 

import shelve # I just need a key value store that is more secure than a dictionary stored entirely in ram

addr = shelve.open('addr', writeback=False) # creates persistent, extremely lightweight database

addr['data'] = {'one':1, 'two': 2}

addr['data2'] = {'one':1, 'two': 2}

print(addr['data']['one'])

#addr.close 

# Works beatifully, but jupyter notebooks breaks it. Lmao

# I can iterate through this dictionary with:

for key in addr:
    print(key, addr[key])
print(addr)

# So addr appears to be a dictionary, with key value pairs. It cannot be reliably iterated through. I'm thinking a  dictionary with the name of the container,
# and relevant values to build the container inside. 

addr.close



In [ ]:
# I might need a way to get ip addressed, but I think I can simply use above code to store used ones, and avoid conflict

In [ ]:
# Now, I need a way to generate the relevant data for the containers. I will use nix's fromJSON feature to get the data from JSON files generated by python

import json






In [17]:
# Actually I have a better Idea. I will use nixs ability to evaluate pure functions and the import feature of nix, which imports text files.

import os

exampleVar = 1

with open("test", "w") as test:
    test.write(str(exampleVar))

# this file can then be evaluated with nix's import function

# this gives me a way to set IP addressed. But I still have no way to set variable names dynamically. 


Okay, here are the nix functions I probably need:

${builtins.readFile ./somefile}" = 4 # readfile, to get file contents as a string, and assign them to a name

import ./somefile # evaluates a nix expression

toString # converts expression or something to string

I think it will be something like:

{
    containers.${builtins.readFile ./name} = {
        stuff, but ip address is important

        ip addr preset
        ip address = "preset,readfile"
    }

    stuff, but packages is important
    I think I have to import the package list from json? Actually, I don't think that will work as json only takes x: y as valid input. 
}

In [28]:
import shelve 

cont = shelve.open('cont')

cont['test'] = {'name':'test', 'addr':"10.250.0"}

for i in cont['test']:
    print(i)
    print(cont['test'][i])
    with open(i, 'w') as file:
        file.write(cont['test'][i])

# Files appear to be clobbered, which is what I want. However, jupyter still breaks my database library. But this will work as a normal python program. 

cont.close

# This database will write the files used by the container.nix config file/program

name
test
addr
10.250.0


<bound method Shelf.close of <shelve.DbfilenameShelf object at 0x7f4e811a62f0>>

In [12]:
# Now I need to generate a hashed password. Nix's mkpasswd feature can do this, but I dunno what it even is. 

# this python program can do it

from getpass import getpass; from crypt import *; \
    p=getpass(); print('\n'+crypt(p, METHOD_SHA512)) \
    if p==getpass('Please repeat: ') else print('\nFailed repeating.')

# However, the crypt library is deprecated, to be removed in python 3.13. Not useful.





$6$1GULzYdJTbdh8cx3$XOnP9zz1pnPOWWifR06fcEMb2S8TRjC9jiI6SaBlrRaA5gAD7WSvvM39tR5zsZBt5vZME1eO8hagPDFCVoY/7.


In [7]:
# from chatgpt:

import random
import hashlib

# Generate a random password of length 12
password = ''.join(random.choices('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789', k=12))

# Generate a SHA-512 hash of the password
salt = hashlib.sha512(str(random.getrandbits(128)).encode('ascii')).hexdigest()[:16]
hash = hashlib.sha512(password.encode('utf-8') + salt.encode('ascii')).hexdigest()

# Format the hash string for /etc/shadow
shadow_hash = f"${{6${salt}}}${{hash}}"

print(password)
print(shadow_hash)


hKTEtknR2VsN
${6$babcc832f61a67ba}${hash}


In [13]:
import hashlib
import os
import binascii

password = "test"

# Generate a random salt
salt = hashlib.sha256(os.urandom(60)).hexdigest().encode('ascii')

# Hash the password with the salt using SHA-512 algorithm
hashed_password = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'), salt, 100000)
hashed_password = binascii.hexlify(hashed_password)

# Combine the salt and hashed password
password_hash = (salt + hashed_password).decode('ascii')

# Format the password hash in the way that Linux stores it in /etc/shadow
password_hash = f"$6${password_hash}"

print(password_hash)


$6$e1ce1e341480dfbf0a6e4dbfed44d7ad0370e044540365d4c9d7999898edd6ed2cc7110b9bdfa2971a08ea3f7ff21d6704271025de54f3dcc9a28641695dd37125cad6da4bd18358f04348589a8c8c977653c3088abe9b113ef9f0c236beeace


In [15]:
# And of course, the lazy option, the linux mkpasswd tool:
# last resort. I want all python, no shell scripts. 

import os

passwd = "test"

print(os.system("mkpasswd" + passwd))

32512


sh: line 1: mkpasswdtest: command not found
